In [94]:
import cv2
import os
import numpy as np
import pickle
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from helpers import resize_to_fit

In [96]:
dados = []
rotulos = []
pasta_base_imagens = 'base_letras'
imagens = paths.list_images(pasta_base_imagens)
for arquivo in imagens:
    rotulo = arquivo.split(os.path.sep)[-2]
    imagem = cv2.imread(arquivo)
    imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2GRAY)
    
    #PADRONIZAR TAMANHO IMAGEM 90X150
    imagem = resize_to_fit(imagem,30,30)
    
    #ADICIONAR DIMENSÃO 2
    imagem = np.expand_dims(imagem,axis=2)
    
    #ADICIONAR A DATA E LABEL
    rotulos.append(rotulo)
    dados.append(imagem)
dados = np.array(dados,dtype="float")/255
rotulos = np.array(rotulos)



In [97]:
#dados de treino e dados de teste

(X_train, X_test, Y_train, Y_test) = train_test_split(dados, rotulos, test_size=0.25)
#converter em on-hot encoding



In [98]:
lb = LabelBinarizer().fit(Y_train)

Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)

# salvar o LabelBinarizer com pickle

with open('rotulos_modelo.dat','wb') as arquivo_pickle:
    pickle.dump(lb, arquivo_pickle)
    

In [99]:
#criar modelo

modelo = Sequential()

# criar as camadas da rede neural
modelo.add(Conv2D(60,(5,5), padding="same", input_shape=(30,30,1), activation="relu"))
modelo.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# criar a 2ª camada
modelo.add(Conv2D(150,(5, 5), padding="same", activation="relu"))
modelo.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# # criar a 3ª camada
modelo.add(Conv2D(375, (5, 5), padding="same", activation="relu"))
modelo.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# # criar a 4ª camada
# modelo.add(Conv2D(625, (5, 5), padding="same", activation="relu"))
# modelo.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# mais uma camada
modelo.add(Flatten())
modelo.add(Dense(1024, activation="relu"))
# camada de saída
modelo.add(Dense(49, activation="softmax"))
# compilar todas as camadas
modelo.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [100]:

# treinar a inteligência artificial
modelo.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=49, epochs=30, verbose=1)



Epoch 1/30
207/207 [==============================] - 46s 219ms/step - loss: 1.9453 - accuracy: 0.4770 - val_loss: 0.3781 - val_accuracy: 0.8814
Epoch 2/30
207/207 [==============================] - 47s 228ms/step - loss: 0.2180 - accuracy: 0.9317 - val_loss: 0.2136 - val_accuracy: 0.9284
Epoch 3/30
207/207 [==============================] - 48s 230ms/step - loss: 0.1037 - accuracy: 0.9671 - val_loss: 0.1561 - val_accuracy: 0.9519
Epoch 4/30
207/207 [==============================] - 48s 230ms/step - loss: 0.0628 - accuracy: 0.9798 - val_loss: 0.1045 - val_accuracy: 0.9709
Epoch 5/30
207/207 [==============================] - 48s 230ms/step - loss: 0.0467 - accuracy: 0.9856 - val_loss: 0.1159 - val_accuracy: 0.9679
Epoch 6/30
207/207 [==============================] - 48s 230ms/step - loss: 0.0306 - accuracy: 0.9915 - val_loss: 0.1055 - val_accuracy: 0.9724
Epoch 7/30
207/207 [==============================] - 48s 230ms/step - loss: 0.0346 - accuracy: 0.9884 - val_loss: 0.2370 - val_ac

In [103]:
# salvar o modelo em um arquivo
modelo.save("modelo_treinado.hdf5")


In [102]:
img = cv2.imread('resolver/imagem.png')

[ WARN:0@54455.759] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('resolver/imagem8600letra1.png'): can't open/read file: check file path/integrity


In [86]:
img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)


In [87]:
img=resize_to_fit(img,30,30)


In [88]:
img = np.expand_dims(img, axis=2)


In [89]:
img = np.expand_dims(img, axis=0)

In [90]:
letra = modelo.predict(img)

In [91]:
letra= lb.inverse_transform(letra)[0]

In [92]:
print(letra)

H


In [93]:
imagens = paths.list_images('letras')

for arquivo in imagens:
    img = cv2.imread(arquivo)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img=resize_to_fit(img,30,30)
    img = np.expand_dims(img, axis=2)
    img = np.expand_dims(img, axis=0)
    letra = modelo.predict(img)
    letra= lb.inverse_transform(letra)[0]
    os.system(f'mv {arquivo} /home/christhian.souza/projects/captcha/base_letras/{letra}')
